# Connecting to and Operations on a Database

## lesson_2_1_2

### Installing PostgreSQL on the Playground Server

At the command line, the first one will require your password:
- `sudo apt update` 
- `sudo apt install postgresql postgresql-contrib`

### Gain Access to PSQL Command Line
- psql is the interactive terminal for working with PostgreSQL

At the command line:
- `sudo -u postgres psql`

You are not logged in as the "postgres" superuser.

### Create User, Database and Grant Access

#### Create Database
- `CREATE DATABASE cloud_user;`

#### Create User
- `CREATE USER cloud_user WITH ENCRYPTED PASSWORD 'cloud_user';`

#### Grant Access to Database by User
- `GRANT ALL PRIVILEGES ON DATABASE cloud_user TO cloud_user;`

You now have a database you can access named cloud_user as the user cloud_user.

#### Leave PSQL
- `\q`

#### Configure PostgreSQL For Remote Access
- PostgreSQL installs with all access to remote users turned off; this is a good thing, think security

To allow remote access:

- `sudo nano /etc/postgresql/10/main/pg_hba.conf`
    
    Add a last entry to the file:
    - `host    all             all             0.0.0.0/0               md5`
- `CTRL-x` to exit and type `Y` and `Enter\Return` to save

- `sudo nano /etc/postgresql/10/main/postgresql.conf`

    Find the section labeled `CONNECTIONS AND AUTHENTICATION` and above this line 
    
    - `#listen_addresses = 'localhost'` add 
    - `listen_addresses = '*'`
- `CTRL-x` to exit and type `Y` and `Enter\Return` to save 

- `sudo systemctl restart postgresql.service`
- To grant access in psql again: `psql -U postgres` postgres sesuaikan dengan nama user yang kalian buat.

### Install Postgresql Driver to Your Virtual Environment
- `conda activate python_data_course`
- `conda install psycopg2`

### **_Start and connect to the Jupyter Notebook server as usual._**

-----

### Server Operations Using Python's Psycopg2

In [4]:
import pandas as pd
import psycopg2

##### A Word About Database Connections

In the cell below you will see I used a `try, except, finally` block.  There are a couple of main reasons and I thought it important to call them out.

- Exceptions:
    Exceptions can occur when trying to connect to a database like PostgreSQL.  Maybe the server is down.  Or perhaps it has already exceeded the maximum number of collections.  It is important to catch those errors and report out to the user (even if it is only you).
    
- Connections:
    There are a limited number of connections a database server can accept.  While this is a ratehr large number, it is possible to reach that number and be refused a connection.  Be a good neighbor and always only open a connection for an operation and then close it.  Don't open a connection and leave it open while you review the data you have gotten.

### Create Table

In [2]:

create_table_query = '''CREATE TABLE source(
    Source varchar (20),
    Target_type varchar (20),
    weight float,
    typeTrans text,
    fraud float4
    );
    CREATE TABLE customer (
    step INTEGER,
    customer varchar(20),
    age varchar(4),
    gender varchar(3),
    zipcode varchar(20),
    merchant varchar(20),
    zipcodemerchant varchar(20),
    category varchar(50),
    amount numeric(10,2),
    fraud int
    );'''
hostStr = '172.18.49.7'
dbPort = 5432
dbStr = 'dbjaka'
uNameStr = 'jaka'
dbPassStr = 'P4ssword'

try:
    # Make connection to db
    cxn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(create_table_query)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')


PostgreSQL connection is closed
Records:
 None


### Add the Data to Table

In [3]:
try: 
    # Make connection to db
    cxn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)
    
    # Create a cursor to db
    cur = cxn.cursor()
    
    with open(r"C:\Users\USER\Downloads\bsNET140513_032310.csv", 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'source', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("table populated")


PostgreSQL connection is closed
table populated


In [4]:
try: 
    # Make connection to db
    cxn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)
    
    # Create a cursor to db
    cur = cxn.cursor()
    
    with open(r"C:\Users\USER\Downloads\bs140513_032310.csv", 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'customer', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("table populated")


PostgreSQL connection is closed
table populated


### Selecting Data From a Server

Use `.fetchall()` with LIMIT or TOP (#) 
- LIMIT works for most databases, but does not work with SQL Server
- TOP (#) is used in place of LIMIT on SQL Server

In [5]:
def db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cur.fetchall()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

Test table is populated by `Select`ing the first five rows.

In [6]:
select_query = '''SELECT * FROM source LIMIT 5;'''

records = db_server_fetch(select_query)
print(records)


Error while connecting to PostgreSQL name 'hostStr' is not defined


UnboundLocalError: cannot access local variable 'cxn' where it is not associated with a value

In [118]:
records

[("'C1093826151'", "'M348934600'", 4.55, "'es_transportation'", 0.0),
 ("'C352968107'", "'M348934600'", 39.68, "'es_transportation'", 0.0),
 ("'C2054744914'", "'M1823072687'", 26.89, "'es_transportation'", 0.0),
 ("'C1760612790'", "'M348934600'", 17.25, "'es_transportation'", 0.0),
 ("'C757503768'", "'M348934600'", 35.72, "'es_transportation'", 0.0)]

In [119]:
def db_server_change(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = conn.commit()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

Add a new record with the following data:
On Saturday, new wait staff Alfred had one person at Breakfast for 10.76 and received a 0.50 tip.

In [131]:
def db_server_change(sql_query):
    records = None
    try:
        # Make connection to the database
        conn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)

        # Create a cursor to the database
        cur = conn.cursor()

        # Send SQL query to the database
        cur.execute(sql_query)
        
        # Commit the transaction
        conn.commit()
        
        # Get the number of affected rows
        records = cur.rowcount

    except psycopg2.Error as error:
        print("Error while connecting to PostgreSQL:", error)

    finally:
        # Closing cursor and database connection
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")

    return records

# Define the SQL query to add data to the database
add_data = '''-- Add a new integer column for age
ALTER TABLE customer
ADD COLUMN age_int INT;

-- Update the new column with converted values
UPDATE customer
SET age_int = age::INT;

-- Once you've verified that all values can be converted, drop the old column and rename the new one
ALTER TABLE customer
DROP COLUMN age,
RENAME COLUMN age_int TO age;
;
'''

add_data2='''
ALTER TABLE tips
ADD COLUMN total_bill_per_person DECIMAL(10, 2),
ADD COLUMN tip_percentage DECIMAL(5, 2),
ADD COLUMN party_size_meal_total_interaction DECIMAL(10, 2);

UPDATE tips
SET total_bill_per_person = meal_total / party_size,
    tip_percentage = (tip / meal_total) * 100,
    party_size_meal_total_interaction = party_size * meal_total;
;
'''

# Call the function to execute the SQL query
affected_rows = db_server_change(add_data)
print("Affected rows:", affected_rows)

Error while connecting to PostgreSQL: syntax error at or near "RENAME"
LINE 12: RENAME COLUMN age_int TO age;
         ^

PostgreSQL connection is closed
Affected rows: None


In [7]:
select_query = '''SELECT * FROM tips LIMIT 5;'''

records = db_server_fetch(select_query)
print(records)

Error while connecting to PostgreSQL name 'hostStr' is not defined


UnboundLocalError: cannot access local variable 'cxn' where it is not associated with a value

In [66]:
df = pd.DataFrame(records, columns=["id", "meal_type", "weekday", "wait_staff", "party_size", "tip","meal_total" ])

# Print the DataFrame
df

ValueError: 7 columns passed, passed data had 10 columns

#### Make a `SELECT` Request to Get New Records

In [20]:
select_query = '''SELECT * FROM tips WHERE wait_staff='Marcia';'''

records = db_server_fetch(select_query)
print(records)

PostgreSQL connection is closed
[(1, 'Saturday', 'Dinner', 'Marcia', 2, 100.64, 16.23), (2, 'Friday', 'Dinner', 'Marcia', 2, 109.84, 5.99), (4, 'Monday', 'Dinner', 'Marcia', 1, 60.01, 8.77), (16, 'Tuesday', 'Breakfast', 'Marcia', 1, 9.71, 1.99), (23, 'Saturday', 'Dinner', 'Marcia', 2, 116.47, 30.94), (26, 'Friday', 'Dinner', 'Marcia', 1, 55.46, 5.29), (33, 'Friday', 'Dinner', 'Marcia', 2, 109.87, 19.47), (43, 'Saturday', 'Dinner', 'Marcia', 4, 249.26, 62.38), (47, 'Friday', 'Dinner', 'Marcia', 3, 187.02, 14.02), (49, 'Friday', 'Dinner', 'Marcia', 2, 113.42, 23.12), (52, 'Friday', 'Dinner', 'Marcia', 2, 118.39, 23.78), (57, 'Saturday', 'Dinner', 'Marcia', 2, 124.83, 19.75), (62, 'Sunday', 'Dinner', 'Marcia', 2, 100.85, 16.84), (68, 'Monday', 'Dinner', 'Marcia', 4, 251.27, 59.05), (69, 'Friday', 'Dinner', 'Marcia', 1, 53.81, 3.45), (73, 'Wednesday', 'Dinner', 'Marcia', 4, 241.81, 47.11), (76, 'Wednesday', 'Breakfast', 'Marcia', 2, 21.55, 2.04), (78, 'Saturday', 'Breakfast', 'Marcia', 4, 

### Accessing a SQL Database With Pandas

pandas.read_sql( ) - loads data from database
pandas.to_sql( ) - write data to database

**CAUTION:** Please don't write to a database unless you know what you are doing and are authorized.  If you are not, your permission should allow read only.

In [21]:
def pandas_db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(host=hostStr, port=dbPort, dbname=dbStr, user=uNameStr, password=dbPassStr)

        # Send sql query to request and create dataframe
        df = pd.read_sql(sql_query, cxn)

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cxn.close()
            print("PostgreSQL connection is closed")
        return df

In [22]:
select_query = '''SELECT * FROM tips WHERE wait_staff='Marcia';'''

alfred_df = pandas_db_server_fetch(select_query)
alfred_df.head()

PostgreSQL connection is closed


C:\Users\USER\AppData\Local\Temp\ipykernel_38568\2124809307.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


,id,weekday,meal_type,wait_staff,party_size,meal_total,tip
0,1,Saturday,Dinner,Marcia,2,100.64,16.23
1,2,Friday,Dinner,Marcia,2,109.84,5.99
2,4,Monday,Dinner,Marcia,1,60.01,8.77
3,16,Tuesday,Breakfast,Marcia,1,9.71,1.99
4,23,Saturday,Dinner,Marcia,2,116.47,30.94


In [23]:
tips_df = pandas_db_server_fetch('''SELECT * FROM tips;''')

PostgreSQL connection is closed


C:\Users\USER\AppData\Local\Temp\ipykernel_38568\2124809307.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


In [38]:
tips_df.tail()

,id,weekday,meal_type,wait_staff,party_size,meal_total,tip
496,497,Saturday,Dinner,Marcia,1,48.71,12.86
497,498,Friday,Dinner,Greg,2,126.26,32.69
498,499,Friday,Dinner,Greg,1,58.99,4.16
499,500,Saturday,Dinner,Peter,2,116.82,21.13
500,504,Saturday,Breakfast,Alfred,1,10.76,0.50


In [37]:
print(tips_df.isnull().sum())

id            0
weekday       0
meal_type     0
wait_staff    0
party_size    0
meal_total    0
tip           0
dtype: int64


In [36]:
print(tips_df.describe())

               id  party_size  meal_total         tip
count  501.000000  501.000000  501.000000  501.000000
mean   251.005988    2.600798  113.749222   17.259002
std    144.780931    1.169752   78.705597   14.295672
min      1.000000    1.000000    4.750000    0.360000
25%    126.000000    2.000000   46.960000    6.210000
50%    251.000000    2.000000  103.460000   13.060000
75%    376.000000    4.000000  181.320000   24.290000
max    504.000000    4.000000  271.370000   63.310000


In [ ]:
df['weekday'] = pd.Categorical(df['weekday'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
df['meal_type'] = pd.Categorical(df['meal_type'])
df['wait_staff'] = pd.Categorical(df['wait_staff'])


In [ ]:
df['weekday'] = pd.to_datetime(df['weekday'])
df['day_of_week'] = df['weekday'].dt.dayofweek  # Monday=0, Sunday=6
df['is_weekend'] = df['weekday'].dt.weekday // 5  # 1 if weekend, 0 if weekday


In [29]:
# Total bill per person
df['total_bill_per_person'] = df['meal_total'] / df['party_size']

# Tip percentage of the total bill
df['tip_percentage'] = (tips_df['tip'] / tips_df['meal_total']) * 100

# Interaction features
df['party_size_meal_total_interaction'] = df['party_size'] * df['meal_total']


In [53]:
df['tip_percentage_2'] = (tips_df['tip'] / tips_df['meal_total']) * 100

In [54]:
#tips_df = tips_df.append(df['tip_percentage'], axis=1)

tips_df = pd.concat([tips_df, pd.DataFrame(df['tip_percentage_2'])], ignore_index=True)

In [56]:
tips_df.tail()

,id,weekday,meal_type,wait_staff,party_size,meal_total,tip,tip_percentage,tip_percentage_2
511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.126789
512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.453387
513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.353591
514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.614231
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.441176


In [48]:
 for col in ['meal_total', 'tip']:
     tips_df[col] = pd.to_numeric(tips_df[col], errors='coerce')

In [47]:
df_int = tips_df.select_dtypes(include=[int])
print(df_int)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[506 rows x 0 columns]


In [39]:
df['party_size_meal_total_interaction'].head(10)

0    32.46
1    11.98
2    88.16
3     8.77
4     1.68
Name: party_size_meal_total_interaction, dtype: float64

In [32]:
df['tip_percentage'] 

0     620.086260
1    1833.722871
2     410.617060
3     684.264538
4     647.619048
Name: tip_percentage, dtype: float64

In [30]:
df['total_bill_per_person']

0    8.115
1    2.995
2    5.510
3    8.770
4    1.680
Name: total_bill_per_person, dtype: float64